In [1]:
import warp as wp
import numpy as np
import trimesh

from warp.sim.collide import TriMeshCollisionDetector

Warp DeprecationWarning: The `warp.sim` module is deprecated and will be removed in v1.10. Please transition to using the forthcoming Newton library instead.


In [2]:
device = wp.get_device('cuda')
# if you don't have a CUDA-compatible GPU try switching to CPU
# device = wp.get_device('cpu')

Warp 1.8.0 initialized:
   CUDA Toolkit 12.8, Driver 12.8
   Devices:
     "cpu"      : "Intel64 Family 6 Model 186 Stepping 2, GenuineIntel"
     "cuda:0"   : "NVIDIA GeForce RTX 3050 6GB Laptop GPU" (6 GiB, sm_86, mempool enabled)
   Kernel cache:
     C:\Users\athuk\AppData\Local\NVIDIA\warp\Cache\1.8.0


In [11]:
mesh = trimesh.load('Cube_subdivided.ply')
# mesh.show()


In [4]:
builder = wp.sim.ModelBuilder()

vertices = [wp.vec3(mesh.vertices[i,:]) for i in range(mesh.vertices.shape[0])]
builder.add_cloth_mesh(
            pos=wp.vec3(0.0, 0.0, 0.0),
            rot=wp.quat_identity(),
            scale=1.0,
            vertices=vertices,
            indices=mesh.faces.reshape(-1),
            vel=wp.vec3(0.0, 0.0, 0.0),
            density=0.02,
            tri_ke=1.0e5,
            tri_ka=1.0e5,
            tri_kd=2.0e-6,
            edge_ke=10,
)
builder.color()
model = builder.finalize()

Module warp.sim.graph_coloring 3c9ae71 load on device 'cpu' took 1.61 ms  (cached)


In [5]:
# to access ForceElementAdjacencyInfo, you need to construct a VBDIntegrator (you dont need to understand what it is)
vbd_integrator = wp.sim.VBDIntegrator(model)

Module warp.sim.integrator_vbd 715ddb2 load on device 'cpu' took 1.22 ms  (cached)


In [6]:
colision_detector = TriMeshCollisionDetector(model)

Module warp.sim.collide 3b78b77 load on device 'cuda:0' took 42.51 ms  (cached)


In [7]:
colision_detector.vertex_triangle_collision_detection(5.0)
colision_detector.edge_edge_collision_detection(5.0)

In [8]:
# d^v_{min}
print(colision_detector.vertex_colliding_triangles_min_dist)
# d^E_{min}
print(colision_detector.edge_colliding_edges_min_dist)
# d^T_{min}
print(colision_detector.triangle_colliding_vertices_min_dist)

[0.08838835 0.08838835 0.08838835 ... 0.08838835 0.08838835 0.08838835]
[0.08838835 0.08838835 0.08838835 ... 0.08838835 0.08838835 0.08838835]
[0.08838835 0.09349301 0.08838835 ... 0.08838835 0.08838835 0.08838835]


In [9]:
from warp.sim.integrator_vbd import get_vertex_num_adjacent_edges, get_vertex_adjacent_edge_id_order, get_vertex_num_adjacent_faces, get_vertex_adjacent_face_id_order, ForceElementAdjacencyInfo
# how to iterate over neighbor elements
@wp.kernel
def iterate_vertex_neighbor_primitives(
    adjacency: ForceElementAdjacencyInfo
):
    particle_idx = wp.tid()

    # iterating over neighbor faces
    num_adj_faces = get_vertex_num_adjacent_faces(adjacency, particle_idx)
    for face_counter in range(num_adj_faces):
        adj_face_idx, vertex_order = get_vertex_adjacent_face_id_order(adjacency, particle_idx, face_counter)
    # iterating over neighbor edges
    num_adj_edges = get_vertex_num_adjacent_edges(adjacency, particle_idx)
    for edge_counter in range(num_adj_edges):
        edge_idx, v_order = get_vertex_adjacent_edge_id_order(adjacency, particle_idx, edge_counter)

wp.launch(
    iterate_vertex_neighbor_primitives,
    dim=model.particle_count,
    inputs=[vbd_integrator.adjacency],
    device=device
)

Module __main__ 3dd20ff load on device 'cuda:0' took 16.01 ms  (cached)


In [49]:
# your turn: you need to:
# Implement conservative bounds computation using the instructions provided above
# it must be implemented using @warp.kernel to maximize efficiency